In [1]:
import os

In [2]:
%pwd

'/Users/_shahid_abdul/Downloads/Programs/My Repositories/Deep-Learning-Kidney-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/_shahid_abdul/Downloads/Programs/My Repositories/Deep-Learning-Kidney-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
# For four Classes
class ConfigurationManager:
   def __init__(
       self,
       config_filepath = CONFIG_FILE_PATH,
       params_filepath = PARAMS_FILE_PATH):

       self.config = read_yaml(config_filepath)
       self.params = read_yaml(params_filepath)

       create_directories([self.config.artifacts_root])

   def get_training_config(self) -> TrainingConfig:
       training = self.config.training
       prepare_base_model = self.config.prepare_base_model
       params = self.params
       training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
       create_directories([
           Path(training.root_dir)
       ])

       training_config = TrainingConfig(
           root_dir=Path(training.root_dir),
           trained_model_path=Path(training.trained_model_path),
           updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
           training_data=Path(training_data),
           params_epochs=params.EPOCHS,
           params_batch_size=params.BATCH_SIZE,
           params_is_augmentation=params.AUGMENTATION,
           params_image_size=params.IMAGE_SIZE
       )

       return training_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
# For four classes
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None

    def get_base_model(self):
        input_shape = tuple(self.config.params_image_size)
        num_classes = 2  # Change to 4 for your case

        self.model = create_model(input_shape, num_classes)
        self.model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    def save_model(self, path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

def create_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')  # Change num_classes to 4
    ])
    return model


In [11]:
# For Four classes
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-04-16 12:13:10,898: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-16 12:13:10,900: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-16 12:13:10,901: INFO: common: created directory at: artifacts]
[2024-04-16 12:13:10,903: INFO: common: created directory at: artifacts/training]
Found 1290 images belonging to 2 classes.
Found 5164 images belonging to 2 classes.
Epoch 1/30


/Users/_shahid_abdul/Downloads/Programs/My Repositories/Deep-Learning-Kidney-Classification/imgclassifier/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


322/322 ━━━━━━━━━━━━━━━━━━━━ 81s 247ms/step - accuracy: 0.7725 - loss: 0.5753 - val_accuracy: 0.7930 - val_loss: 0.4567
Epoch 2/30
322/322 ━━━━━━━━━━━━━━━━━━━━ 0s 181us/step - accuracy: 0.8750 - loss: 0.3420 - val_accuracy: 0.0000e+00 - val_loss: 1.7728
Epoch 3/30


2024-04-16 12:14:36.799067: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-04-16 12:14:36.845775: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


322/322 ━━━━━━━━━━━━━━━━━━━━ 86s 264ms/step - accuracy: 0.8150 - loss: 0.4098 - val_accuracy: 0.8125 - val_loss: 0.3699
Epoch 4/30
322/322 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 0.6875 - loss: 0.5394 - val_accuracy: 0.9000 - val_loss: 0.7905
Epoch 5/30


...

2024-04-16 12:39:02.342138: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-16 12:39:02.404689: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


322/322 ━━━━━━━━━━━━━━━━━━━━ 106s 326ms/step - accuracy: 0.9758 - loss: 0.0648 - val_accuracy: 0.9305 - val_loss: 0.1609
Epoch 28/30
322/322 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 1.0000 - loss: 0.0311 - val_accuracy: 0.9000 - val_loss: 0.1526
Epoch 29/30


2024-04-16 12:40:48.618077: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-16 12:40:48.666018: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


322/322 ━━━━━━━━━━━━━━━━━━━━ 97s 299ms/step - accuracy: 0.9551 - loss: 0.1481 - val_accuracy: 0.9336 - val_loss: 0.1468
Epoch 30/30
322/322 ━━━━━━━━━━━━━━━━━━━━ 0s 177us/step - accuracy: 1.0000 - loss: 0.0114 - val_accuracy: 1.0000 - val_loss: 0.0313
[2024-04-16 12:42:26,025: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


2024-04-16 12:42:25.968362: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-16 12:42:26.017809: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
